# Recomendador de peliculas

In [1]:
import pandas as pd
import numpy as np
import sqlite3



pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [2]:
url_movies = 'https://raw.githubusercontent.com/4GeeksAcademy/k-nearest-neighbors-project-tutorial/main/tmdb_5000_movies.csv'

url_credits = 'https://raw.githubusercontent.com/4GeeksAcademy/k-nearest-neighbors-project-tutorial/main/tmdb_5000_credits.csv'

In [3]:
df_movies = pd.read_csv(url_movies)

df_credits = pd.read_csv(url_credits)

df_movies.to_csv('../data/raw/movies.csv')
df_credits.to_csv('../data/raw/credits.csv')

In [4]:
df_movies.sample(5)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
3147,9500000,"[{""id"": 27, ""name"": ""Horror""}, {""id"": 878, ""na...",NaN,106845,"[{""id"": 3974, ""name"": ""reality tv""}, {""id"": 10...",en,Re-Kill,"Five years after a zombie outbreak, the men an...",5.346205,"[{""name"": ""Signature Pictures"", ""id"": 2982}, {...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2015-10-16,0,87.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,We are the endangered species.,Re-Kill,4.9,34
1333,37000000,"[{""id"": 18, ""name"": ""Drama""}]",http://www.magnoliamovie.com/,334,"[{""id"": 455, ""name"": ""farewell""}, {""id"": 802, ...",en,Magnolia,An epic mosaic of many interrelated characters...,30.993736,"[{""name"": ""New Line Cinema"", ""id"": 12}, {""name...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",1999-12-08,48451803,188.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Things fall down. People look up. And when it ...,Magnolia,7.5,825
1276,40000000,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 35, ""name...",http://www.thousandwordsmovie.com/,72358,"[{""id"": 9468, ""name"": ""liar""}, {""id"": 11204, ""...",en,A Thousand Words,"Jack McCall is a fast-talking literary agent, ...",13.119133,"[{""name"": ""Paramount Pictures"", ""id"": 4}, {""na...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,22044277,91.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,He only has 1000 words left to discover what m...,A Thousand Words,6.0,373
3483,6000000,"[{""id"": 35, ""name"": ""Comedy""}]",NaN,16172,"[{""id"": 1946, ""name"": ""restaurant""}, {""id"": 20...",en,Simply Irresistible,"After her mother's death, mediocre chef Amanda...",5.696491,"[{""name"": ""New Regency Pictures"", ""id"": 10104}...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",1999-02-05,4398989,95.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,Magic opened up their hearts... Love did the r...,Simply Irresistible,5.5,52
4481,0,"[{""id"": 99, ""name"": ""Documentary""}]",NaN,2287,"[{""id"": 542, ""name"": ""street gang""}, {""id"": 89...",en,Rize,A documentary film that highlights two street ...,1.623769,"[{""name"": ""David LaChapelle Studios"", ""id"": 10...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2005-01-15,0,86.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,NaN,Rize,5.9,13


In [5]:
df_credits.sample(5)

,movie_id,title,cast,crew
4486,302579,Naturally Native,"[{""cast_id"": 0, ""character"": ""Vickie Lewis Big...","[{""credit_id"": ""5462d90c0e0a2677fd00116d"", ""de..."
4457,905,Pandora's Box,"[{""cast_id"": 7, ""character"": ""Lulu"", ""credit_i...","[{""credit_id"": ""52fe428ac3a36847f8026e81"", ""de..."
3724,270938,Falcon Rising,"[{""cast_id"": 0, ""character"": ""John 'Falcon' Ch...","[{""credit_id"": ""537cc3a70e0a26316200067c"", ""de..."
1319,87421,Riddick,"[{""cast_id"": 1, ""character"": ""Riddick"", ""credi...","[{""credit_id"": ""52fe49c49251416c910b8163"", ""de..."
2717,142,Brokeback Mountain,"[{""cast_id"": 11, ""character"": ""Ennis Del Mar"",...","[{""credit_id"": ""55659add92514135430014da"", ""de..."


Voy a comprobar si mis titulos son unicos, las instrucciones indican que deberia hacer un join a traves de la columna titulo, pero si hay algun titulo repetido, esto borraria y solo dejaria uno.

In [6]:
df_movies.shape

(4803, 20)

In [7]:
df_movies['title'].nunique()

4800

In [8]:
repeated_titles = df_credits['title'][df_credits['title'].duplicated()].unique()
print(repeated_titles)

['The Host' 'Out of the Blue' 'Batman']


Veo que si que tengo duplicados, voy a comprobar las filas a ver si son duplicados completos

In [9]:
resultado = df_movies[df_movies['title'] == 'Batman']
resultado

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
1359,35000000,"[{""id"": 14, ""name"": ""Fantasy""}, {""id"": 28, ""na...",NaN,268,"[{""id"": 848, ""name"": ""double life""}, {""id"": 84...",en,Batman,The Dark Knight of Gotham City begins his war ...,44.104469,"[{""name"": ""PolyGram Filmed Entertainment"", ""id...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",1989-06-23,411348924,126.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Have you ever danced with the devil in the pal...,Batman,7.0,2096
4267,1377800,"[{""id"": 10751, ""name"": ""Family""}, {""id"": 12, ""...",NaN,2661,"[{""id"": 339, ""name"": ""submarine""}, {""id"": 849,...",en,Batman,The Dynamic Duo faces four super-villains who ...,9.815394,"[{""name"": ""Twentieth Century Fox Film Corporat...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",1966-07-30,0,105.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,He's Here Big As Life In A Real Bat-Epic,Batman,6.1,203


In [10]:
resultado = df_movies[df_movies['title'] == 'Out of the Blue']
resultado

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
3647,0,"[{""id"": 18, ""name"": ""Drama""}]",NaN,39269,"[{""id"": 4470, ""name"": ""punk""}, {""id"": 10183, ""...",en,Out of the Blue,Dennis Hopper is a hard-drinking truck driver ...,0.679351,"[{""name"": ""Robson Street"", ""id"": 71953}]","[{""iso_3166_1"": ""CA"", ""name"": ""Canada""}]",1980-05-01,0,94.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,A harrowing drama from the director of Easy Rider,Out of the Blue,6.5,17
3693,0,"[{""id"": 18, ""name"": ""Drama""}]",NaN,10844,"[{""id"": 2658, ""name"": ""new zealand""}, {""id"": 3...",en,Out of the Blue,Ordinary people find extraordinary courage in ...,0.706355,[],"[{""iso_3166_1"": ""NZ"", ""name"": ""New Zealand""}]",2006-10-12,0,103.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The true story of a small town massacre,Out of the Blue,5.9,18


In [11]:
resultado = df_movies[df_movies['title'] == 'The Host']
resultado

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
972,44000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",NaN,72710,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,The Host,A parasitic alien soul is injected into the bo...,42.933027,"[{""name"": ""Nick Wechsler Productions"", ""id"": 8...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2013-03-22,63327201,125.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,You will be one of us,The Host,6.0,1817
2877,11000000,"[{""id"": 27, ""name"": ""Horror""}, {""id"": 18, ""nam...",http://www.hostmovie.com/,1255,"[{""id"": 1261, ""name"": ""river""}, {""id"": 1880, ""...",ko,괴물,Gang-du is a dim-witted man working at his fat...,27.655270,"[{""name"": ""Cineclick Asia"", ""id"": 685}, {""name...","[{""iso_3166_1"": ""KR"", ""name"": ""South Korea""}]",2006-07-27,88489643,119.0,"[{""iso_639_1"": ""ko"", ""name"": ""\ud55c\uad6d\uc5...",Released,Monsters are real.,The Host,6.7,537


Puedo observar que son completamente diferentes, por lo tanto unir por titulo no seria factible, ya que estos repetidos se eliminarian

Voy a comprobar duplicados en la columna ID, en caso de que no los haya sera la que utilice para unir mis dos datasets

In [12]:
print(df_movies['id'].nunique())
print(df_credits['movie_id'].nunique())

4803
4803


Puedo observar que no hay duplicados en los ids de cada uno de mis datasets, intentare hacer el join a traves de ellos, en caso de obtener mas filas de las que tengo originalmente, sera porque no coinciden entre si

In [13]:
# Creo mi database y la conexion a ella
conn = sqlite3.connect('../data/processed/movie_database.db')

# Creo el objeto cursor que interactuara con ella
cursor = conn.cursor()

# Creo mis tablas
df_movies.to_sql('tabla_movies', conn, if_exists='replace', index=False)
df_credits.to_sql('tabla_credits', conn, if_exists='replace', index=False)

4803

In [14]:
query = """
CREATE TABLE IF NOT EXISTS tabla_resultados AS 
SELECT tabla_credits.movie_id, tabla_movies.title, tabla_movies.overview, 
       tabla_movies.genres, tabla_movies.keywords, tabla_credits.cast, tabla_credits.crew
FROM tabla_movies
INNER JOIN tabla_credits
ON tabla_movies.id = tabla_credits.movie_id;
"""


cursor.execute(query)
conn.commit()

In [15]:
df = pd.read_sql('select * from tabla_resultados', conn)

conn.close()

In [16]:
df.sample(5)

,movie_id,title,overview,genres,keywords,cast,crew
2791,59457,Womb,A woman's consuming love forces her to bear th...,"[{""id"": 10749, ""name"": ""Romance""}, {""id"": 878,...",[],"[{""cast_id"": 3, ""character"": ""Rebecca"", ""credi...","[{""credit_id"": ""591009b8c3a36864d402c9c1"", ""de..."
819,35056,Date Night,Phil and Claire Foster fear that their mild-ma...,"[{""id"": 35, ""name"": ""Comedy""}]","[{""id"": 248, ""name"": ""date""}, {""id"": 417, ""nam...","[{""cast_id"": 2, ""character"": ""Claire Foster"", ...","[{""credit_id"": ""5839e689c3a3681cdc00be61"", ""de..."
1653,76640,The Last Stand,Ray Owens is sheriff of the quiet US border to...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 798, ""name"": ""sheriff""}, {""id"": 1415, ...","[{""cast_id"": 2, ""character"": ""Sheriff Ray Owen...","[{""credit_id"": ""534e53950e0a2679e0001588"", ""de..."
4326,94329,The Raid,Deep in the heart of Jakarta's slums lies an i...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 53, ""nam...","[{""id"": 9730, ""name"": ""crime boss""}, {""id"": 14...","[{""cast_id"": 11, ""character"": ""Rama"", ""credit_...","[{""credit_id"": ""52fe49529251416c750c441d"", ""de..."
3383,33676,Losin' It,"Set in 1965, four Los Angeles school friends -...","[{""id"": 35, ""name"": ""Comedy""}]","[{""id"": 255, ""name"": ""male nudity""}, {""id"": 29...","[{""cast_id"": 1, ""character"": ""Woody"", ""credit_...","[{""credit_id"": ""52fe452e9251416c9102a291"", ""de..."


In [17]:
df.shape

(4803, 7)

Con esto he conseguido unir las dos bases de datos sin eliminar las filas repetidas.

nearest neighbours, tf-idf

In [18]:
import json

df = df.assign(genres=lambda x: x['genres'].apply(lambda y: [item['name'] for item in json.loads(y)] if pd.notna(y) else None))
df = df.assign(keywords=lambda x: x['keywords'].apply(lambda y: [item['name'] for item in json.loads(y)] if pd.notna(y) else None))
df = df.assign(cast=lambda x: x['cast'].apply(lambda y: [item['name'] for item in json.loads(y)][:3] if pd.notna(y) else None))

# columna crew
df = df.assign(crew=lambda x: x['crew'].apply(lambda y: next((item['name'] for item in json.loads(y) if item.get('job') == 'Director'), None) if pd.notna(y) else None))

df = df.assign(overview=lambda x: x['overview'].apply(lambda y: y.split() if pd.notna(y) else []))

In [19]:
df.head()

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",James Cameron
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[Johnny Depp, Orlando Bloom, Keira Knightley]",Gore Verbinski
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send...","[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...","[Daniel Craig, Christoph Waltz, Léa Seydoux]",Sam Mendes
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney...","[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...","[Christian Bale, Michael Caine, Gary Oldman]",Christopher Nolan
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili...","[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...","[Taylor Kitsch, Lynn Collins, Samantha Morton]",Andrew Stanton


In [22]:
df.sample(5)

,movie_id,title,overview,genres,keywords,cast,crew
4408,18869,Jimmy and Judy,"[Two, misunderstood, suburban, kids, challenge...","[Action, Crime, Drama, Thriller]","[police, revenge, independent film, video came...","[Edward Furlong, Rachael Bella, William Sadler]",Randall Rubin
3417,7347,Elite Squad,"[In, 1997,, before, the, visit, of, the, pope,...","[Drama, Action, Crime]","[slum, police brutality, brazilian, war on dru...","[Wagner Moura, Caio Junqueira, André Ramiro]",José Padilha
214,2133,The Perfect Storm,"[In, October, 1991,, a, confluence, of, weathe...",[Drama],"[u.s. air force, grocery, jamaican, meteorolog...","[George Clooney, Mark Wahlberg, Diane Lane]",Wolfgang Petersen
2751,5876,The Mist,"[After, a, violent, storm,, a, dense, cloud, o...","[Science Fiction, Horror, Thriller]","[father son relationship, monster, supermarket...","[Thomas Jane, Marcia Gay Harden, Laurie Holden]",Frank Darabont
2131,16052,Selena,"[In, this, biographical, drama,, Selena, Quint...",[Drama],"[musical, tejano, elopement, bustier, fan club]","[Jennifer Lopez, Edward James Olmos, Constance...",Gregory Nava
